# Recognition of Yoga Postures

## Necessary libraries
We choose
[Pytorch](https://pytorch.org/)
as our framework for this project for its learning parh and It is Simple GPU installation.
And we are going to work with

In [1]:
import os
import shutil
import numpy as np
import random
import shutil
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from shutil import copyfile
import matplotlib.pyplot as plt
from keras.callbacks import ModelCheckpoint
from utils import split_data
from fastai.vision.all import get_image_files,verify_images
import gradio as gr
from sklearn.metrics import confusion_matrix
import seaborn as sns
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.losses import categorical_crossentropy,SparseCategoricalCrossentropy
from tensorflow.keras.optimizers.legacy import Adam
import datetime

/Users/johanjairgilcesreyes/miniconda3/envs/myenv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# check if CUDA is available to use GPU on the training 
train_on_gpu = tf.test.is_gpu_available()

if train_on_gpu:
    print('GPU con soporte CUDA disponible')
else:
    print('No se encontró GPU con soporte CUDA')

In [ ]:
images=get_image_files('../YOGA/dataset')
images
failed = verify_images(images)
failed

In [ ]:
images=get_image_files('../YOGA/DATA')
images
failed = verify_images(images)
failed

In [ ]:
images=get_image_files('../YOGA/dataset/test/')
images
failed = verify_images(images)
failed

In [ ]:
# Define paths
downdog_SRC_DIR = "data/downdog"
goddess_SRC_DIR = "data/goddess"
plank_SRC_DIR = "data/plank"
tree_SRC_DIR = "data/tree"
warrior_SRC_DIR = "data/warrior"
TRAINING_DIR = "dataset/training"
VALIDATION_DIR = "dataset/validation"
TEST_DIR = "dataset/test"
#splitting the dataset
TRAINING_DDG_DIR = os.path.join(TRAINING_DIR, "downdog/")
VALIDATION_DDG_DIR = os.path.join(VALIDATION_DIR, "downdog/")
TEST_DDG_DIR = os.path.join(TEST_DIR, "downdog/")

TRAINING_GDS_DIR = os.path.join(TRAINING_DIR, "goddess/")
VALIDATION_GDS_DIR = os.path.join(VALIDATION_DIR, "goddess/")
TEST_GDS_DIR = os.path.join(TEST_DIR, "goddess/")

TRAINING_PLANK_DIR = os.path.join(TRAINING_DIR, "plank/")
VALIDATION_PLANK_DIR = os.path.join(VALIDATION_DIR, "plank/")
TEST_PLANK_DIR = os.path.join(TEST_DIR, "plank/")

TRAINING_TREE_DIR = os.path.join(TRAINING_DIR, "tree/")
VALIDATION_TREE_DIR = os.path.join(VALIDATION_DIR, "tree/")
TEST_TREE_DIR = os.path.join(TEST_DIR, "tree/")

TRAINING_WR_DIR = os.path.join(TRAINING_DIR, "warrior/")
VALIDATION_WR_DIR = os.path.join(VALIDATION_DIR, "warrior/")
TEST_WR_DIR = os.path.join(TEST_DIR, "warrior/")
# Define proportion of images used for training
train_size = 0.8
val_size = 0.1
test_size = 0.1

split_data(downdog_SRC_DIR,
           TRAINING_DDG_DIR,
           VALIDATION_DDG_DIR,
           TEST_DDG_DIR,
           split_ratio=(train_size,val_size, test_size))

split_data(goddess_SRC_DIR,
           TRAINING_GDS_DIR,
           VALIDATION_GDS_DIR,
           TEST_GDS_DIR,
           split_ratio=(train_size,val_size, test_size))

split_data(plank_SRC_DIR,
           TRAINING_PLANK_DIR,
           VALIDATION_PLANK_DIR,
           TEST_PLANK_DIR,
           split_ratio=(train_size,val_size, test_size))

split_data(tree_SRC_DIR,
           TRAINING_TREE_DIR,
           VALIDATION_TREE_DIR,
           TEST_TREE_DIR,
           split_ratio=(train_size,val_size, test_size))
split_data(warrior_SRC_DIR,
           TRAINING_WR_DIR,
           VALIDATION_WR_DIR,
           TEST_WR_DIR,
           split_ratio=(train_size,val_size, test_size))

In [ ]:
print(f"Original Downdog directory has {len(os.listdir(downdog_SRC_DIR))} images")
print(f"Original Goddess directory has {len(os.listdir(goddess_SRC_DIR))} images")
print(f"Original Plank directory has {len(os.listdir(plank_SRC_DIR))} images")
print(f"Original Tree directory has {len(os.listdir(tree_SRC_DIR))} images")
print(f"Original Warrior directory has {len(os.listdir(warrior_SRC_DIR))} images\n")

# Training and validation splits
print(f"There are {len(os.listdir(TRAINING_DDG_DIR))} images of Dogdown for training")
print(f"There are {len(os.listdir(TRAINING_GDS_DIR))} images of Goddess for training")
print(f"There are {len(os.listdir(TRAINING_PLANK_DIR))} images of Plank for training")
print(f"There are {len(os.listdir(TRAINING_TREE_DIR))} images of Tree for training")
print(f"There are {len(os.listdir(TRAINING_WR_DIR))} images of Warrior for training\n")

print(f"There are {len(os.listdir(VALIDATION_DDG_DIR))} images of Dogdown for validation")           
print(f"There are {len(os.listdir(VALIDATION_GDS_DIR))} images of Goddess for validation")
print(f"There are {len(os.listdir(VALIDATION_PLANK_DIR))} images of Plank for validation")
print(f"There are {len(os.listdir(VALIDATION_TREE_DIR))} images of Tree for validation")
print(f"There are {len(os.listdir(VALIDATION_WR_DIR))} images of Warrior for validation\n")

print(f"There are {len(os.listdir(TEST_DDG_DIR))} images of Dogdown for test")
print(f"There are {len(os.listdir(TEST_GDS_DIR))} images of Goddess for test")
print(f"There are {len(os.listdir(TEST_PLANK_DIR))} images of Plank for test")
print(f"There are {len(os.listdir(TEST_TREE_DIR))} images of Plank for test")
print(f"There are {len(os.listdir(TEST_WR_DIR))} images of tree for test")

## Data


In [ ]:
# Definir un generador de imágenes con transformaciones
batch_size = 16
width=250
height=250
data_generator = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.20
)


# Cargar imágenes del directorio y aplicar transformaciones
train_data = data_generator.flow_from_directory(
    TRAINING_DIR,
    target_size=(width, height),
    batch_size=batch_size,
    class_mode='sparse',  
    color_mode="rgb"
)

validation_data = data_generator.flow_from_directory(
    VALIDATION_DIR,
    target_size=(width, height),
    batch_size=batch_size,
    color_mode="rgb",
    class_mode='sparse',
)

test_data = data_generator.flow_from_directory(
    TEST_DIR,
    target_size=(width, height),
    batch_size=batch_size,
    color_mode="rgb",
    class_mode='sparse',
)


In [ ]:
for data, labels in train_data:
    print(labels)
    # Aquí puedes agregar más código para inspeccionar los datos y etiquetas
    break  # Detenemos el ciclo después de imprimir las etiquetas de un lote


In [ ]:
train_data

Observemos una imagen 

In [ ]:
# Obtener un lote de imágenes y etiquetas del generador de datos
images, labels = next(train_data)

# Etiquetas de las clases (nombre de las carpetas)
class_labels = list(train_data.class_indices.keys())
print(class_labels)
# Configurar subplots
num_images = 8
num_cols = 3 # Número de columnas en la cuadrícula
num_rows = (num_images + num_cols - 1) // num_cols  # Calcular el número de filas necesarias

# Crear una cuadrícula de subplots
fig, axes = plt.subplots(num_rows, num_cols, figsize=(10, 5))

# Iterar a través de las imágenes y subplots
for i, ax in enumerate(axes.flatten()):
    if i < num_images:
        image = images[i]
        label = class_labels[int(labels[i])]
        
        ax.imshow(image)
        ax.set_title(f"Clase: {label}")
        ax.axis('off')
    else:
        ax.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
class Net(models.Model):
    def __init__(self):
        super(Net, self).__init__()

        # Primera serie de capas convolucionales y pooling
        self.conv1_1 = layers.Conv2D(32, kernel_size=3, padding='same', activation='relu')
        self.conv1_2 = layers.Conv2D(32, kernel_size=3, padding='same', activation='relu')
        self.bn1 = layers.BatchNormalization()
        self.pool1 = layers.MaxPooling2D(pool_size=(2, 2))

        # Segunda serie de capas convolucionales y pooling
        self.conv2_1 = layers.Conv2D(64, kernel_size=3, padding='same', activation='relu')
        self.conv2_2 = layers.Conv2D(64, kernel_size=3, padding='same', activation='relu')
        self.bn2 = layers.BatchNormalization()
        self.pool2 = layers.MaxPooling2D(pool_size=(2, 2))

        # Tercera serie de capas convolucionales y pooling
        self.conv3_1 = layers.Conv2D(128, kernel_size=3, padding='same', activation='relu')
        self.conv3_2 = layers.Conv2D(128, kernel_size=3, padding='same', activation='relu')
        self.bn3 = layers.BatchNormalization()
        self.pool3 = layers.MaxPooling2D(pool_size=(2, 2))

        # Cuarta serie de capas convolucionales y pooling
        self.conv4_1 = layers.Conv2D(256, kernel_size=3, padding='same', activation='relu')
        self.conv4_2 = layers.Conv2D(256, kernel_size=3, padding='same', activation='relu')
        self.bn4 = layers.BatchNormalization()
        self.pool4 = layers.MaxPooling2D(pool_size=(2, 2))

        self.flatten = layers.Flatten()

        # Capas densas
        self.fc1 = layers.Dense(512, activation='relu')
        self.dropout1 = layers.Dropout(0.3)
        self.fc2 = layers.Dense(256, activation='relu')
        self.dropout2 = layers.Dropout(0.3)
        self.fc3 = layers.Dense(128, activation='relu')
        self.dropout3 = layers.Dropout(0.3)
        self.fc4 = layers.Dense(5, activation='softmax')

    def call(self, x):
        x = self.pool1(self.bn1(self.conv1_2(self.conv1_1(x))))
        x = self.pool2(self.bn2(self.conv2_2(self.conv2_1(x))))
        x = self.pool3(self.bn3(self.conv3_2(self.conv3_1(x))))
        x = self.pool4(self.bn4(self.conv4_2(self.conv4_1(x))))
        x = self.flatten(x)
        x = self.dropout1(self.fc1(x))
        x = self.dropout2(self.fc2(x))
        x = self.dropout3(self.fc3(x))
        x = self.fc4(x)
        return x



In [ ]:
class ImprovedNet(models.Model):
    def __init__(self):
        super(ImprovedNet, self).__init__()

        # Primera serie de capas convolucionales y pooling
        self.conv1_1 = layers.Conv2D(32, kernel_size=3, padding='same', activation='relu')
        self.conv1_2 = layers.Conv2D(32, kernel_size=3, padding='same', activation='relu')
        self.dropout1 = layers.Dropout(0.2)
        self.pool1 = layers.MaxPooling2D(pool_size=(2, 2))

        # Segunda serie de capas convolucionales y pooling
        self.conv2_1 = layers.Conv2D(64, kernel_size=3, padding='same', activation='relu')
        self.conv2_2 = layers.Conv2D(64, kernel_size=3, padding='same', activation='relu')
        self.dropout2 = layers.Dropout(0.2)
        self.pool2 = layers.MaxPooling2D(pool_size=(2, 2))

        # Tercera serie de capas convolucionales y pooling
        self.conv3_1 = layers.Conv2D(128, kernel_size=3, padding='same', activation='relu')
        self.conv3_2 = layers.Conv2D(128, kernel_size=3, padding='same', activation='relu')
        self.dropout3 = layers.Dropout(0.2)
        self.pool3 = layers.MaxPooling2D(pool_size=(2, 2))

        # Cuarta serie de capas convolucionales y pooling
        self.conv4_1 = layers.Conv2D(256, kernel_size=3, padding='same', activation='relu')
        self.conv4_2 = layers.Conv2D(256, kernel_size=3, padding='same', activation='relu')
        self.dropout4 = layers.Dropout(0.2)
        self.pool4 = layers.MaxPooling2D(pool_size=(2, 2))

        # Quinta serie de capas convolucionales y pooling
        self.conv5_1 = layers.Conv2D(512, kernel_size=3, padding='same', activation='relu')
        self.conv5_2 = layers.Conv2D(512, kernel_size=3, padding='same', activation='relu')
        self.dropout5 = layers.Dropout(0.2)
        self.pool5 = layers.MaxPooling2D(pool_size=(2, 2))

        self.flatten = layers.Flatten()

        # Capas densas
        self.fc1 = layers.Dense(64, activation='relu')
        self.fc_dropout1 = layers.Dropout(0.3)
        self.fc2 = layers.Dense(32, activation='relu')
        self.fc_dropout2 = layers.Dropout(0.3)
        self.fc3 = layers.Dense(5, activation='softmax')

    def call(self, x):
        x = self.pool1(self.dropout1(self.conv1_2(self.conv1_1(x))))
        x = self.pool2(self.dropout2(self.conv2_2(self.conv2_1(x))))
        x = self.pool3(self.dropout3(self.conv3_2(self.conv3_1(x))))
        x = self.pool4(self.dropout4(self.conv4_2(self.conv4_1(x))))
        x = self.pool5(self.dropout5(self.conv5_2(self.conv5_1(x))))
        x = self.flatten(x)
        x = self.fc_dropout1(self.fc1(x))
        x = self.fc_dropout2(self.fc2(x))
        x = self.fc3(x)
        return x


In [ ]:

class TransferNet(models.Model):
    def __init__(self, base_model_weights=None):
        super(TransferNet, self).__init__()
        self.base_model = tf.keras.applications.VGG16(include_top=False, weights=base_model_weights, input_shape=(250, 250, 3))
        self.flatten = layers.Flatten()
        # Capas densas
        self.fc1 = layers.Dense(64, activation='relu')
        self.fc_dropout1 = layers.Dropout(0.3)
        self.fc2 = layers.Dense(32, activation='relu')
        self.fc_dropout2 = layers.Dropout(0.3)
        self.fc3 = layers.Dense(5, activation='softmax')

    def call(self, x):
        x = self.base_model(x)
        x = self.flatten(x)
        x = self.fc_dropout1(self.fc1(x))
        x = self.fc_dropout2(self.fc2(x))
        x = self.fc3(x)
        return x
    def get_config(self):
        return {"base_model_name": "VGG16"}


    @classmethod
    def from_config(cls, config):
        if config["base_model_name"] == "VGG16":
            return cls(base_model_weights="imagenet")




In [ ]:
class ModifiedTransferNetVGG(models.Model):
    def __init__(self, base_model_weights=None):
        super(ModifiedTransferNetVGG, self).__init__()
        self.base_model = tf.keras.applications.VGG16(include_top=False, weights=base_model_weights, input_shape=(250, 250, 3))
        self.flatten = layers.Flatten()
        
        # Capa convolucional adicional
        self.extra_conv = layers.Conv2D(512, kernel_size=3, padding='same', activation='relu')
        self.extra_pool = layers.MaxPooling2D(pool_size=(2, 2))
        self.extra_dropout = layers.Dropout(0.3)
        
        # Capas densas
        self.fc1 = layers.Dense(128, activation='relu')
        self.fc_dropout1 = layers.Dropout(0.4)
        self.fc2 = layers.Dense(64, activation='relu')
        self.fc_dropout2 = layers.Dropout(0.4)
        self.fc3 = layers.Dense(32, activation='relu')
        self.fc_dropout3 = layers.Dropout(0.4)
        self.fc4 = layers.Dense(5, activation='softmax')

    def call(self, x):
        x = self.base_model(x)
        x = self.extra_dropout(self.extra_pool(self.extra_conv(x)))
        x = self.flatten(x)
        x = self.fc_dropout1(self.fc1(x))
        x = self.fc_dropout2(self.fc2(x))
        x = self.fc_dropout3(self.fc3(x))
        x = self.fc4(x)
        return x


In [ ]:
class ModifiedTransferNetResNet(models.Model):
    def __init__(self, base_model_weights=None):
        super(ModifiedTransferNetResNet, self).__init__()
        self.base_model = tf.keras.applications.ResNet50(include_top=False, weights=base_model_weights, input_shape=(250, 250, 3))
        self.flatten = layers.Flatten()
        
        # Capa convolucional adicional
        self.extra_conv = layers.Conv2D(512, kernel_size=3, padding='same', activation='relu')
        self.extra_pool = layers.MaxPooling2D(pool_size=(2, 2))
        self.extra_dropout = layers.Dropout(0.3)
        
        # Capas densas
        self.fc1 = layers.Dense(128, activation='relu')
        self.fc_dropout1 = layers.Dropout(0.4)
        self.fc2 = layers.Dense(64, activation='relu')
        self.fc_dropout2 = layers.Dropout(0.4)
        self.fc3 = layers.Dense(32, activation='relu')
        self.fc_dropout3 = layers.Dropout(0.4)
        self.fc4 = layers.Dense(5, activation='softmax')

    def call(self, x):
        x = self.base_model(x)
        x = self.extra_dropout(self.extra_pool(self.extra_conv(x)))
        x = self.flatten(x)
        x = self.fc_dropout1(self.fc1(x))
        x = self.fc_dropout2(self.fc2(x))
        x = self.fc_dropout3(self.fc3(x))
        x = self.fc4(x)
        return x


In [ ]:
#model = Net()
#model = ImprovedNet()
model = TransferNet()


In [ ]:
criterion = SparseCategoricalCrossentropy()
optimizer = Adam(learning_rate=0.0004230253)
#optimizer = tf.keras.optimizers.legacy.RMSprop(learning_rate=0.001)

EPOCHS=20

In [ ]:
print(len(train_data),len(validation_data))

In [ ]:
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)
lr_schedule = tf.keras.callbacks.LearningRateScheduler(scheduler)


In [ ]:
model.compile(optimizer=optimizer, 
              loss=SparseCategoricalCrossentropy(), 
              metrics=['accuracy'])

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, verbose=0, restore_best_weights=True)
checkpoint = tf.keras.callbacks.ModelCheckpoint('checkpoint', monitor='val_loss', verbose=0, save_best_only=True)
#callbacks = [early_stopping, checkpoint, lr_schedule]
callbacks = [early_stopping, checkpoint]

In [ ]:
STEP_SIZE_TRAIN=train_data.n//train_data.batch_size
STEP_SIZE_VALID=validation_data.n//validation_data.batch_size
STEP_SIZE_TEST=test_data.n//test_data.batch_size
STEP_SIZE_TRAIN

In [ ]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
callbacks = [early_stopping, checkpoint, tensorboard_callback]


In [ ]:
history =model.fit(train_data,
    step_per_epoch=STEP_SIZE_TRAIN,
    validation_data=validation_data,
    epochs=EPOCHS,
    callbacks=callbacks
)

In [ ]:
loss_train = history.history['loss']
loss_val = history.history['val_loss']
epochs = range(1,EPOCHS+1)
plt.plot(epochs, loss_train, 'g', label='Training loss')
plt.plot(epochs, loss_val, 'b', label='validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()
    
loss_train = history.history['accuracy']
loss_val = history.history['val_accuracy']
epochs = range(1,EPOCHS+1)
plt.plot(epochs, loss_train, 'g', label='Training accuracy')
plt.plot(epochs, loss_val, 'b', label='validation accuracy')
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()



In [ ]:
model.summary()

In [ ]:
model.save_weights('yoga_weights.h5')

In [ ]:
#for gradio
def predict_image(img):
    # Redimensionar y preprocesar la imagen
    img = img.reshape(1, width, height, 3)
    img = img / 255.0

    # Hacer la predicción
    predictions = model.predict(img)
    predicted_class = np.argmax(predictions)

    # Obtener el nombre de la clase
    class_names = list(train_data.class_indices.keys())
    return class_names[predicted_class]


In [ ]:


interface = gr.Interface(
    fn=predict_image, 
    inputs=gr.inputs.Image(shape=(width, height)), 
    outputs="text",
    live=True,
    capture_session=True
)
interface.launch()


### Resultados del modelo

In [ ]:
# Obtener las predicciones del modelo
predictions = model.predict(test_data)
predicted_classes = np.argmax(predictions, axis=1)

# Obtener las etiquetas verdaderas
true_classes = test_data.classes
class_labels = list(test_data.class_indices.keys())

# Calcular la matriz de confusión
confusion = confusion_matrix(true_classes, predicted_classes)

# Visualizar la matriz de confusión
plt.figure(figsize=(10, 8))
sns.heatmap(confusion, annot=True, cmap='Blues', fmt='g', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
from sklearn.metrics import classification_report
import numpy as np

# 1. Predice las etiquetas de tus datos de prueba.
predictions = model.predict(test_data)
predicted_classes = np.argmax(predictions, axis=1)

# 2. Obtén las etiquetas verdaderas de tus datos de prueba.
true_classes = test_data.classes
class_labels = list(test_data.class_indices.keys())

# 3. Usa `classification_report` para obtener el informe.
report = classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report)


In [ ]:
# Asegúrate de que el modelo ha sido llamado al menos una vez
dummy_input = np.random.random((1, width, height, 3))
_ = model(dummy_input)


## Fine tuning

In [ ]:
for layer in base_model.layers[-5:]:
    layer.trainable = True

# Compilar el modelo con un learning rate muy bajo
optimizer_fine_tuning = tf.keras.optimizers.Adam(learning_rate=0.00001)
model.compile(optimizer=optimizer_fine_tuning, 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(), 
              metrics=['accuracy'])

# Continuar el entrenamiento
history =model.fit(train_data,
        steps_per_epoch=STEP_SIZE_TRAIN,
        validation_data=validation_data,
        validation_steps=STEP_SIZE_VALID,
        epochs=EPOCHS,
        callbacks=callbacks
)

In [ ]:
loss_train = history.history['loss']
loss_val = history.history['val_loss']
epochs = range(1,EPOCHS+1)
plt.plot(epochs, loss_train, 'g', label='Training loss')
plt.plot(epochs, loss_val, 'b', label='validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()
    
loss_train = history.history['accuracy']
loss_val = history.history['val_accuracy']
epochs = range(1,EPOCHS+1)
plt.plot(epochs, loss_train, 'g', label='Training accuracy')
plt.plot(epochs, loss_val, 'b', label='validation accuracy')
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()



## Searching best HiperParameters


In [ ]:
from kerastuner import HyperModel

class TransferNetTuner(HyperModel):
    def __init__(self, base_model_weights=None):
        self.base_model_weights = base_model_weights


In [ ]:
def build_model(hp):
        model = TransferNet(base_model_weights='imagenet')
        
       # Hiperparámetro para el learning rate
        lr = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG', default=1e-3)

        # Hiperparámetro para el optimizador
        optimizer = hp.Choice('optimizer', ['adam', 'sgd', 'rmsprop'])

        # Compilación del modelo
        if optimizer == 'adam':
                opt = tf.keras.optimizers.legacy.Adam(learning_rate=lr)
        elif optimizer == 'sgd':
                opt = tf.keras.optimizers.SGD(learning_rate=lr)
        elif optimizer == 'rmsprop':
                opt = tf.keras.optimizers.RMSprop(learning_rate=lr)

        model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        return model

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_loss',       # Métrica a monitorizar. En este caso, es la pérdida de validación.
    patience=10,              # Número de épocas sin mejora después de las cuales se detiene el entrenamiento.
    verbose=1,                # Para imprimir mensajes.
    restore_best_weights=True # Restaura los pesos del modelo al mejor punto encontrado durante el entrenamiento.
)

In [ ]:
from kerastuner.tuners import RandomSearch

# Usar solo la mitad de los datos para acelerar la búsqueda

half_steps_per_epoch = int(STEP_SIZE_TRAIN / 2)

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='random_search_dir',
    project_name='project_name_here'
)


tuner.search(train_data,
             validation_data=validation_data,
            steps_per_epoch=half_steps_per_epoch,
             epochs=5,
             callbacks=[early_stopping])


In [ ]:
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]
print(best_hyperparameters.values)
#{'learning_rate': 0.004230253117597585}